## Fleet and workforce planning with Linear Programming
#### A preview of Linear Programming's capabilities to solve some business problems

**Problem:** finding the optimal (less costly) way of transporting product boxes from warehouses to stores employing different vans and workers while fulfilling the constraints of demand, stock, vans and workers assignment.

**Context Variables**:
- $p\in E$: products
- $w\in P$: warehouses
- $s\in S$: stores
- $v \in V$: vans
- $e \in E$: employees
- $(e_1, e_2) \in E \times E$: conflicting pairs of employees

**Assumptions**:
- We know the demand in boxes of each of our products, by store;
- We know how many boxes of our products are available in each of our warehouses;
- All the boxes have the same size (to simplify the problem);
- We know how many boxes fit in each of the vehicles;
- The cost of sending a box varies depending on the combination of warehouse-store;
- The use of each van has a fixed cost;
- The vans can't go from a warehouse to a store without transporting at least 1 box;
- The vans can't do more than 5 trips. We can assume that they all occur on the same day;
- There's a limit to how many boxes can be transported by each van;
- None of the vans can repeat the same trip warehouse-store;
- We need 2 employees assigned per van to be able to make a trip;
- If we assign an employee we have to pay him for this task;
- Each employee can only be assigned to a single van;
- It is desirable to avoid assigning conflicting employees to the same vehicle.

### Variables, Objective and Constraints

- $c_{pwsv} \in (0,+\infty)$: Unit cost of sending a box of product $p$ from warehouse $w$ to store $s$ using vehicle $v$
- $x_{pwsv} \in [0,+\infty)$: Number of boxes of product $p$ sent from warehouse $w$ to store $s$ using van $v$
- $\delta_v \in \{0,1\}$: Boolean variable created to verifiy that the vehicle $v$ is used
- $f_v \in (0,+\infty)$: Integer fixed fee for using the vehicle $v$
- $\epsilon_{ve} \in \{0,1\}$: Boolean variable created to verify if an employee $e$ is assigned to a van $v$
- $t_{wsv} \in \{0,1\}$: Boolean auxiliary variable used to check if a trip from warehouse $w$ to store $s$ was made using van $v$
- $\alpha_{vj} \in \{0,1\}$: Boolean auxiliary variable to check if just one employee of a conflicting pair $(e_1, e_2)$ is assigned to a van $v$
- $\beta_{vj} \in \{0,1\}$: Boolean variable to check if the conflicting pair $(e_1, e_2)$ is assigned to the same van $v$

$\min \sum c_{pwsv}x_{pwsv}+ \sum\limits_{v\in V} t_{v}f_{v}+ \sum\limits_{v\in V \\ e \in E} 1500 \epsilon_{ve}+ \sum\limits_{v\in V\\ j \in J}500\beta_{vj} \space (1)$

subject to
 
- $\sum\limits_{\substack{w \in W \\ v \in V}} x_{pwsv} = demand_{ps} \space \forall p \in P, s \in S \space (2)$
- $\sum\limits_{\substack{s \in S \\ v \in V}}x_{pwsv} \le stock_{pw} \space \forall p \in P, w \in W \space (3)$
- $\sum\limits_{\substack{p\in P}} x_{pwsv} \le capacity_{v} t_{wsv} \space \forall w\in W, s\in S, v\in V \space (4)$
- $\sum\limits_{\substack{w \in W \\ s \in S}} t_{wsv} \le 5\delta_v \space \forall v \in V \space (5)$
- $\sum\limits_{\substack{e \in E}}\epsilon_{ve} = 2\delta_v \space \forall v\in V \space (6)$
- $\sum\limits_{\substack{v \in V}} \epsilon_{ve} \le 1 \space \forall e \in E \space (7)$
- $\sum\limits_{\substack{v \in V}} \epsilon_{ve_1} + \epsilon_{ve_2} - \alpha_{vj} - 2\beta_{vj} = 0 \space \forall e\in E, j\in J \space \textnormal{with} \space J=\{(e_a,e_b) \in E \times E: \textnormal{pairs with conflicts} \} \space (8)$

#### Solution

In [ ]:
import numpy as np
import pandas as pd
from itertools import product
from ortools.sat.python import cp_model

: 

Create random data from hyperparameters in order to replicate bussiness-as-usual scenario

In [99]:
# Specify a seed to be able to replicate the experiment
np.random.seed(9)

# Declare the context variables
number_warehouses = 17
number_stores = 10
number_products = 1
number_vehicles = 4
number_employees = 8
trip_limit = 12

# Set some thresholds for the simulated data
min_stock, max_stock = 40, 50
min_demand, max_demand = 50, 100
min_capacity, max_capacity = 15, 20

# thresholds for costs
min_cost, max_cost = 1, 6
min_van_fixed_cost, max_van_fixed_cost = 100, 200
fixed_salary = 150
conflict_penalty = 50

# Generate the data:
# Cost matrix per product (of size number_warehouses x number_stores x number_vehicles)
# Stock vector per product (of size number_warehouses)
# Demand vector per product (of size number_stores)
# Capacity list per van (i.e. how many boxes each can transport)
# List of pairs of conflicting employees
# Vector of fixed vans fees if the vehicle is used

capacities = [25, 20, 20, 15]
conflicted_employees = [
    (0,1),(0,2),(0,3),(0,4),(0,5),(0,6),(0,7),
    (1,2),(1,3),(1,4),(1,5),(1,6),(1,7),
    (2,3),(2,4),(2,5),(2,6),(2,7),
    (3,4),(3,5),(3,6),(3,7),
    (4,5),(4,6),(4,7),
]

# For each product we'll generate random costs, stocks and demands, given by the intervals [low,high]
costs_shipping = np.random.randint(min_cost, max_cost, size=(number_warehouses, number_stores))
stocks = np.random.randint(min_stock, max_stock, size=(number_products, number_warehouses))
demands = np.random.randint(min_demand, max_demand, size=(number_products, number_stores))
van_fees = np.random.randint(min_van_fixed_cost, max_van_fixed_cost, size=number_vehicles)


Call instance of OR-tools' SAT solver and declare variables to solve mixed interger poblems

In [100]:
model = cp_model.CpModel()
solver = cp_model.CpSolver()

# Variable reflects the number of boxes shipped from a warehouse to a store using a specific van
box = {}
for p in range(number_products):
    for w in range(number_warehouses):
        for s in range(number_stores):
            for v in range(number_vehicles):
                # Write the variables following the previous mathematical notation x[Product,Warehouse,Store, Van]
                # Here we also specify that each quantity must be positive (from 0 to infinity)
                box[p, w, s, v] = model.NewIntVar(0, max(capacities), f"box_{p}_{w}_{s}_{v}")

# Variable reflects the use of a van 
is_vehicle_used = {}
for v in range(number_vehicles):
    is_vehicle_used[v] = model.NewBoolVar(f"is_used_{v}")

# Variable signals if employee e is assigned to van v
is_employee_assigned = {}
for v in range(number_vehicles):
    for e in range(number_employees):
        is_employee_assigned[v, e] = model.NewBoolVar(f"is_assigned_{v}_{e}")

# This is an auxiliary variable used to count the number of trips a van makes
trip = {}
for v in range(number_vehicles):
    for w in range(number_warehouses):
        for s in range(number_stores):
            trip[w, s, v] = model.NewBoolVar(f"trip_{w}_{s}_{v}")

# Auxiliary variable to denotate if a vans is conflicted by two employees traveling along
is_vehicle_conflicted = {}
for v in range(number_vehicles):
    for e1, e2 in conflicted_employees:
        is_vehicle_conflicted[v, e1, e2] = model.NewBoolVar(f"is_conflicted_{v}_{e1}_{e2}")

Define constrains

In [101]:
# The sum of the columns (stock received at each store) must equal the store's demand
for (p, s) in product(range(number_products), range(number_stores)):
    pairs = product(range(number_warehouses), range(number_vehicles))
    boxes_shipped_to_store = sum(box[p, _w, s, _v] for (_w, _v) in pairs)
    model.Add(boxes_shipped_to_store == demands[p, s]) # product_demand_in_store

# The sum of stock of product p received at each store must be lower or equal to the available stock of p at each warehouse w
for (p, w) in product(range(number_products), range(number_warehouses)):
    pairs = product(range(number_stores), range(number_vehicles))
    boxes_shipped_from_warehouse = sum(box[p, w, _s, _v] for (_s, _v) in pairs)
    model.Add(boxes_shipped_from_warehouse <= stocks[p, w])  # product stock in warehouse

# None of the vans can make more than 5 trips
# a) Register the possibility of making a trip from warehouse w to store s using van v
for v in range(number_vehicles):
    for s in range(number_stores):
        for w in range(number_warehouses):
            boxes_shipped_on_vehicle = sum(box[_p, w, s, v] for _p in range(number_products))
            model.AddLinearConstraint(boxes_shipped_on_vehicle, 1, capacities[v]).OnlyEnforceIf(trip[w, s, v])
            model.Add(boxes_shipped_on_vehicle == 0).OnlyEnforceIf(trip[w, s, v].Not())
            
# b) Limit the number of trips each van can make and check that a van was used
for v in range(number_vehicles):
    trips_per_vehicle = sum(trip[_w, _s, v] for (_w, _s) in product(range(number_warehouses), range(number_stores)))
    model.AddLinearConstraint(trips_per_vehicle, 1, trip_limit).OnlyEnforceIf(is_vehicle_used[v])
    model.Add(trips_per_vehicle == 0).OnlyEnforceIf(is_vehicle_used[v].Not())

    # The number of employees assigned to a van needs to 2 or 0
    employees_in_vehicle = sum(is_employee_assigned[v, _e] for _e in range(number_employees))
    model.Add(employees_in_vehicle == 2).OnlyEnforceIf(is_vehicle_used[v])
    model.Add(employees_in_vehicle == 0).OnlyEnforceIf(is_vehicle_used[v].Not())

    #Check which pair of employees are conflicted in order to pay the penalty
    for e1, e2 in conflicted_employees:
        partners = [is_employee_assigned[v, e1], is_employee_assigned[v, e2]]
        model.AddBoolAnd(partners).OnlyEnforceIf(is_vehicle_conflicted[v, e1, e2])

        not_partners = [is_employee_assigned[v, e1].Not(), is_employee_assigned[v, e2].Not()]
        model.AddBoolOr(not_partners).OnlyEnforceIf(is_vehicle_conflicted[v, e1, e2].Not())

# An employee assigned to a van can't be assigned to another van
for e in range(number_employees):
    vehicle_options_for_employee = sum(is_employee_assigned[_v, e] for _v in range(number_vehicles))
    model.Add(vehicle_options_for_employee <= 1)

Define the objective function previously mentioned


In [102]:
objective_function = []

# First term -> sum of variable costs of transportation
for p in range(number_products):
    for w in range(number_warehouses):
        for s in range(number_stores):
            for v in range(number_vehicles):
                objective_function.append(costs_shipping[w, s] * box[p, w, s, v])

# Second term -> sum of fixed costs of using a specific van
for v in range(number_vehicles):
    objective_function.append(van_fees[v] * is_vehicle_used[v])

# Third term -> sum of salary payments per employee assigned to a task
for v in range(number_vehicles):
    for e in range(number_employees):
        objective_function.append(fixed_salary * is_employee_assigned[v, e])

# Fourth term -> sum of penalties for not avoiding the joint assignment of conflicting employees
for v, e1, e2 in is_vehicle_conflicted.keys():
    objective_function.append(conflict_penalty * is_vehicle_conflicted[v, e1, e2])

# Specify the type of problem. In this case, we want to minimize the objective function
solver.parameters.num_search_workers = 8
solver.parameters.max_time_in_seconds = 60
model.Minimize(sum(objective_function))

Trigger solver and watch solutions come out

In [103]:
# Call the solver method to find the optimal solution
callback = cp_model.ObjectiveSolutionPrinter()
or_status = solver.SolveWithSolutionCallback(model, callback)
status = solver.StatusName(or_status)

if status in ["OPTIMAL", "FEASIBLE"]:
    print(f'Solution: Total cost = ${solver.ObjectiveValue()} dollars')
else:
    print('A solution could not be found, check the problem specification')

Solution 0, time = 0.45 s, objective = 3623
Solution 1, time = 0.54 s, objective = 3582
Solution 2, time = 2.48 s, objective = 2786
Solution 3, time = 3.57 s, objective = 2782
Solution 4, time = 4.65 s, objective = 2779
Solution 5, time = 4.74 s, objective = 2756
Solution 6, time = 26.81 s, objective = 2491
Solution 7, time = 27.87 s, objective = 2441
Solution 8, time = 40.45 s, objective = 2416
Solution 9, time = 43.31 s, objective = 2412
Solution 10, time = 43.97 s, objective = 2398
Solution 11, time = 44.91 s, objective = 2372
Solution 12, time = 47.63 s, objective = 2357
Solution 13, time = 48.82 s, objective = 2333
Solution 14, time = 57.19 s, objective = 2330
Solution: Total cost = $2330.0 dollars


Extract results and assert if constrains are fulfilled

In [104]:
data = []
for (p, w, s, v), or_var in box.items():
    n = or_var.Name()
    x = solver.Value(or_var)
    data.append([n, x, p, w, s, v])

names = [
    "name",
    "value",
    "prod",
    "ware",
    "store",
    "vehic",
]

df = pd.DataFrame(data, columns=names)

# Test if demand is fulfilled by store in all products
for (p, s) in product(range(number_products), range(number_stores)):
    total = df[(df['value'] > 0) & (df["store"] == s) & (df["prod"] == p)]
    assert demands[p][s] == total.sum(numeric_only=True).value, "Error"

#All demand fulfilled of product 1 in store 0
df[(df['value'] > 0) & (df["store"] == 0) & (df["prod"] == 0)]

,name,value,prod,ware,store,vehic
320,box_0_8_0_0,25,0,8,0,0
321,box_0_8_0_1,19,0,8,0,1
560,box_0_14_0_0,24,0,14,0,0
642,box_0_16_0_2,11,0,16,0,2


In [95]:
# Vans to be used
vans = {}
for v, or_var in is_vehicle_used.items():
    vans[v] = bool(solver.Value(or_var))
    print(v, vans[v])

0 True
1 True
2 True
3 False


In [96]:
# Trips
data = []
for (w, s, v), or_var in trip.items():
    n = or_var.Name()
    x = bool(solver.Value(or_var))
    data.append([n, x, w, s, v])

names = [
    "name",
    "value",
    "ware",
    "store",
    "vehic",
]
df_trips = pd.DataFrame(data, columns=names)

# Assert if total number of trips are below the limits per vehicle
for v in range(number_vehicles):
    total = df_trips[df_trips["value"] & (df_trips["vehic"] == v)].sum(numeric_only=True)
    assert (total.value == 0) if not vans[v] else (total.value <= trip_limit), "Error"

# Trips performed by vehicle 0
df_trips[df_trips["value"] & (df_trips["vehic"] == 0)]


,name,value,ware,store,vehic
9,trip_0_9_0,True,0,9,0
13,trip_1_3_0,True,1,3,0
24,trip_2_4_0,True,2,4,0
46,trip_4_6_0,True,4,6,0
49,trip_4_9_0,True,4,9,0
54,trip_5_4_0,True,5,4,0
78,trip_7_8_0,True,7,8,0
91,trip_9_1_0,True,9,1,0
109,trip_10_9_0,True,10,9,0
117,trip_11_7_0,True,11,7,0


In [97]:
# Employees assignation
data = []
for (v, e), or_var in is_employee_assigned.items():
    n = or_var.Name()
    x = bool(solver.Value(or_var))
    data.append([n, x, v, e])

names = [
    "name",
    "value",
    "vehic",
    "emplo",
]
df_assign = pd.DataFrame(data, columns=names)

# Assert if number of employees assigned to vehicle is 2 or 0
for v in range(number_vehicles):
    total = df_assign[df_assign["value"] & (df_assign["vehic"] == v)].sum(numeric_only=True)
    assert total.value == 2 if vans[v] else total.value == 0, "Error"

#All assignations of employees to their vehicles
df_assign[df_assign["value"]]

,name,value,vehic,emplo
0,is_assigned_0_0,True,0,0
2,is_assigned_0_2,True,0,2
13,is_assigned_1_5,True,1,5
15,is_assigned_1_7,True,1,7
20,is_assigned_2_4,True,2,4
22,is_assigned_2_6,True,2,6


In [98]:
# Conflicts
for (v, e1, e2), or_var in is_vehicle_conflicted.items():
    if solver.Value(or_var):
        print(f"vehicle {v} is conflicted by employees {e1}, {e2}")

vehicle 0 is conflicted by employees 0, 2
vehicle 2 is conflicted by employees 4, 6
